In [1]:
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
import os
import fiona
import pandas as pd
import sys
#%matplotlib inline  # jupyter "magic" to display plots in notebook
pd.set_option('mode.chained_assignment', None)
import numpy as np
from bs4 import BeautifulSoup
import re
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from geojson import Point, Feature, FeatureCollection, dump

import sys
sys.path.append("../")
import src.soporteScrapping as scr

parameters

In [2]:
url = 'https://download.geofabrik.de/europe/italy/'
country = 'IT'
country_clean = 'IT'

get links

In [3]:
links,flattened = scr.getLinks(url)
links

200


['https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip']

In [4]:
flattened

['centro-latest-free.shp.zip',
 'isole-latest-free.shp.zip',
 'nord-est-latest-free.shp.zip',
 'nord-ovest-latest-free.shp.zip',
 'sud-latest-free.shp.zip']

data conversion table input

In [5]:
ruta = os.path.dirname(os.getcwd()) + '\\data'

In [6]:
inPOIS = pd.ExcelFile(f'{ruta}\\POIS_Classification' + '.xlsx')
sheets_list = inPOIS.sheet_names[0]
POIS = pd.read_excel(inPOIS, sheet_name = sheets_list)
POIS.rename({'Tipo2':'Subtipo', 'Subtipo':'fclass'}, axis = 1, inplace=True)
POIS.head()

c:\Users\annas\anaconda3\envs\iron\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


downloading shapefile

In [7]:
print('Downloading shapefile...')
patron = r'(.*).shp.zip'
for link, flat in zip(links, flattened):
    local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content), mode='a')
    folder = re.findall(patron, flat)
    os.mkdir(os.path.join(local_path, folder[0]))
    z.extractall(path=(local_path + '//'+ folder[0])) # extract to folder
    print(f"Done with {link}")
filenames = [y for y in sorted(z.namelist()) for starting in ['gis_osm_landuse_a_free_1', 'gis_osm_buildings_a_free_1'] if y.startswith(starting)] 
print(filenames)

Done with https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip
['gis_osm_buildings_a_free_1.cpg', 'gis_osm_buildings_a_free_1.dbf', 'gis_osm_buildings_a_free_1.prj', 'gis_osm_buildings_a_free_1.shp', 'gis_osm_buildings_a_free_1.shx', 'gis_osm_landuse_a_free_1.cpg', 'gis_osm_landuse_a_free_1.dbf', 'gis_osm_landuse_a_free_1.prj', 'gis_osm_landuse_a_free_1.shp', 'gis_osm_landuse_a_free_1.shx']


POIs

In [8]:
local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'

In [9]:
alldf_pois = []
for subdir, dirs, files in os.walk(local_path):
    #global alldf_landuse
    for file in files:
        #print(file)
        if file.startswith('gis_osm_pois_free_1.shp'):
            print(file, subdir)
            df_pois = gpd.read_file(subdir + '\\' + 'gis_osm_pois_free_1' + '.shp')
            alldf_pois.append(df_pois)

gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\centro-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\isole-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-est-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-ovest-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\sud-latest-free


In [13]:
df_pois = pd.concat([(alldf_pois[n]) for n in list(range(len(alldf_pois)))])

In [14]:
print(df_pois.shape)
df_pois.head()

(727951, 5)


,osm_id,code,fclass,name,geometry
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276)
1,26859360,2422,camp_site,Albinia,POINT (10.52806 43.01722)
2,26859373,2422,camp_site,Argentario Camping Village,POINT (11.19245 42.49649)
3,26859377,2422,camp_site,Camping Village Assisi,POINT (12.57420 43.07560)
4,26859385,2422,camp_site,Badiaccia,POINT (12.01639 43.18028)


reproject

In [17]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [25]:
df_pois.to_crs("EPSG:4326", inplace=True)

In [19]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

spatial join geoLevels

In [24]:
GeoLevels_IT_03 = gpd.read_file(ruta + '\\' + 'GeoLevels_IT_03.geojson')
GeoLevels_IT_03.to_crs("EPSG:4326", inplace = True)

In [26]:
df_pois_merge = df_pois.sjoin(GeoLevels_IT_03, how="inner")
print(df_pois_merge.shape)
df_pois_merge.head(1)

(717159, 16)


,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano


merge with equivalence table

In [27]:
POIS.head()

,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


In [28]:
df_pois_merge = df_pois_merge.merge(POIS, on ='fclass', how = 'left', indicator = True)
df_pois_merge['_merge'].value_counts()

both          717158
left_only          1
right_only         0
Name: _merge, dtype: int64

In [30]:
df_pois_merge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 717159 entries, 0 to 717158
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   osm_id         717159 non-null  object  
 1   code           717159 non-null  int64   
 2   fclass         717159 non-null  object  
 3   name           330894 non-null  object  
 4   geometry       717159 non-null  geometry
 5   index_right    717159 non-null  int64   
 6   OBJECTID       717159 non-null  int64   
 7   nivel          717159 non-null  int64   
 8   cod_Pais       717159 non-null  object  
 9   nom_Pais       717159 non-null  object  
 10  cod_Region     717159 non-null  object  
 11  nom_Region     717159 non-null  object  
 12  cod_Provincia  717159 non-null  object  
 13  nom_Provincia  717159 non-null  object  
 14  cod_Municipio  717159 non-null  object  
 15  nom_Municipio  717159 non-null  object  
 16  Tipo           408585 non-null  object  
 17  Su

In [31]:
df_pois_merge.head()

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,_merge
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,Art & Culture,Archeological,both
1,417059088,2907,camera_surveillance,NaN,POINT (11.64423 42.65732),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,NaN,NaN,both
2,456875080,2004,post_box,NaN,POINT (11.64599 42.65771),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,NaN,NaN,both
3,610578172,2742,viewpoint,San Rocco,POINT (11.71067 42.68410),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,Tourism,Art & Culture,both
4,741597346,2901,toilet,NaN,POINT (11.64302 42.65776),4078,1238525,3,IT,Italia,9,Toscana,53,Grosseto,53026,Sorano,Amenity,Amenity,both


selecting only the data that falls into sicily

In [32]:
df_pois_merge['nom_Region'].unique()

array(['Toscana', 'Umbria', 'Lazio', 'Marche', 'Emilia-Romagna', 'Puglia',
       'Liguria', 'Abruzzo', 'Molise', 'Campania', 'Sardegna', 'Sicilia',
       'Trentino-Alto Adige', 'Veneto', 'Friuli Venezia Giulia',
       'Lombardia', 'Piemonte', "Valle d'Aosta", 'Calabria', 'Basilicata'],
      dtype=object)

In [33]:
df_pois_merge = df_pois_merge[df_pois_merge['nom_Region'] == 'Sicilia']

In [35]:
df_pois_merge.drop(['_merge'], axis = 1, inplace = True)

export

In [34]:
local_path = os.path.dirname(os.getcwd()) + f'\\data'
local_path

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data'

In [41]:
df_pois_merge['lon'] = df_pois_merge.geometry.apply(lambda p: p.x)
df_pois_merge['lat'] = df_pois_merge.geometry.apply(lambda p: p.y)
df_pois_merge.head(1)

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,lon,lat
127938,26860774,2422,camp_site,Camping Biscione,POINT (12.47814 37.70137),6936,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,12.478139,37.701373


In [42]:
df_pois_merge.to_file(f'{local_path}\\df_pois.geojson', driver='GeoJSON')  

In [37]:
df_pois_merge.head(1)

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo
127938,26860774,2422,camp_site,Camping Biscione,POINT (12.47814 37.70137),6936,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality


In [39]:
df_pois_merge.to_csv(f'{local_path}//pois.csv', encoding='utf-8', sep='|', decimal='.', index=False)

In [40]:
print(df_pois_merge[df_pois_merge['nom_Region'] == 'Sicilia'].shape)

(33079, 20)
